In [1]:
import lxml.html
from pathlib import Path
import pickle
import re
import time
from tqdm import tqdm

from projectmir.xmldocument import XMLDocument
from projectmir.functions import find_identifier_definition

In [2]:
# p_path = './data/test_latexml/'
# p = Path(p_path)
# x = list(p.glob('*.html'))
# documents_IDs = [x_.name[:-5] for x_ in x]

# doc = [[]]*len(documents_IDs)
# for i, documents_ID in tqdm(enumerate(documents_IDs)):
#     documents_path = p_path + documents_ID + '.html'
#     doc_ = find_identifier_definition(documents_path)
#     doc[i] = doc_

# save_filename = 'doc' + time.strftime('_%d%b%Y_%H%M%S') + '.pkl'
# with open(save_filename, 'wb') as f:
#     pickle.dump(doc, f)


In [3]:
load_filename = 'doc.pkl'
with open(load_filename, 'rb') as f:
    doc = pickle.load(f)
print(doc[0].path)
print(doc[0].identifiers)

./data/test_latexml/test_sample.html
['w_i', 'T_i', 'Q', 'T', 'w', 'V', 'ρ', 'C', 'U_int', 'H', '\\overset{^}{U}_int', '\\overset{^}{H}', 'd', 't', 'T_ref', '\\overset{^}{H}_ref', '\\overset{^}{H}_i', 'Δ', 'N_F']


In [81]:
def is_identifier(math_txt):
    not_number = math_txt[0] not in [1, 2, 3, 4, 5, 6, 7, 8, 9, 0]
    return not_number


def tree_to_str(ml_tree):
    ml_str = ''
    ml_tree_ = copy.deepcopy(ml_tree)
    if isinstance(ml_tree_, lxml.html.HtmlElement):
        if ml_tree_.tag in ['mi', 'mo', 'mn']:
            # ml_str += ml_tree_.text_content()
            math_txt = ml_tree_.text_content()

        else:
            for ml_children in ml_tree_:
                ml_children_ = [x for x in ml_children]
                if ml_children.tag in ['mi', 'mo', 'mn']:
                    math_txt = ml_children.text_content() + 'x'

                elif ml_tree_.tag == 'mrow':
                    math_txt = tree_to_str(ml_children)

                elif ml_tree_.tag == 'msubsup':
                    math_txt = tree_to_str(ml_children_[0]) + '_' + tree_to_str(ml_children_[1]) \
                                + '^' + tree_to_str(ml_children_[2])

                elif ml_tree_.tag == 'msub':
                    math_txt = tree_to_str(ml_children_[0]) + '_' + tree_to_str(ml_children_[1])

                elif ml_children.tag == 'msup':
                    math_txt = tree_to_str(ml_children_[0]) + '^' + tree_to_str(ml_children_[1])

                elif ml_children.tag == 'munderover':
                    math_txt = r'\overset{' + tree_to_str(ml_children_[2]) + '}{' + r'\underset{' + \
                                tree_to_str(ml_children_[1]) + '}{' + tree_to_str(ml_children_[0]) + '}}'

                elif ml_tree_.tag == 'mover':
                    math_txt = r'\overset{' + \
                                tree_to_str(ml_children_[1]) + '}{' + tree_to_str(ml_children_[0]) + '}'

                elif ml_children.tag == 'munder':
                    math_txt = r'\underset{' + \
                                tree_to_str(ml_children_[1]) + '}{' + tree_to_str(ml_children_[0]) + '}'

                else:
                    math_txt = ''
                    warnings.warn('unexpected tag')
                    print(f'{ml_tree}')

                # ml_str += math_txt

    elif isinstance(ml_tree_, list):
        tree_to_str(ml_tree)

    else:
        warnings.warn('unexpected tag')
        print(f'{ml_tree}')

    return ml_str

def extract_ml_component(
        html_cssselect_math,
        mltag,
        identifiers,
        reg_string_list):
    identifiers_ = list(identifiers)
    reg_string_list_ = list(reg_string_list)
    for html_math_mltag in html_cssselect_math.cssselect(mltag):
        # if mltag == 'mi':
        #     math_txt = html_math_mltag.text_content()
        #     identifier_candidate = html_math_mltag
        # else:
        #     html_math_mltag_component = [
        #         x for x in html_math_mltag.iterchildren()]
        #     identifier_candidate = html_math_mltag_component[0]
        #     math_txt = [x.text_content()
        #                 for x in html_math_mltag_component]
        #     if mltag == 'msubsup':
        #         math_txt = math_txt[0] + '_' + \
        #             math_txt[1] + '^' + math_txt[2]
        #     elif mltag == 'msub':
        #         math_txt = math_txt[0] + '_' + math_txt[1]
        #     elif mltag == 'msup':
        #         math_txt = math_txt[0] + '^' + math_txt[1]
        #     elif mltag == 'munderover':
        #         math_txt = r'\overset{' + math_txt[2] + '}{' + \
        #             r'\underset{' + math_txt[1] + '}{' + math_txt[0] + '}}'
        #     elif mltag == 'mover':
        #         math_txt = r'\overset{' + \
        #             math_txt[1] + '}{' + math_txt[0] + '}'
        #     elif mltag == 'munder':
        #         math_txt = r'\underset{' + \
        #             math_txt[1] + '}{' + math_txt[0] + '}'
        math_txt = tree_to_str(html_math_mltag)


        if is_identifier(math_txt) and (
                math_txt not in identifiers_):
            identifiers_.append(math_txt)
        # drop_tree is for extracting w_i without w and i.
        html_math_mltag.drop_tree()
        reg_string = lxml.html.tostring(
            html_math_mltag, encoding='unicode')
        if is_identifier(math_txt):
            reg_string_list_.append(
                (math_txt, reg_string, 'MATH{:04d}'.format(
                    identifiers_.index(math_txt))))
        else:
            reg_string_list_.append(
                (math_txt, reg_string, math_txt))
    return identifiers_, reg_string_list_

In [82]:
# x_text = """
# <p class="ltx_p">Next, we show how the general energy balance in Eq. 2-10 can be simplified for this particular example.For a pure liquid at low or moderate pressures, the internal energy is approximately equal to the enthalpy, <math id="S1.SS1.p2.m1" class="ltx_Math" alttext="U_{\rm int}\approx H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>≈</mo><mi>H</mi></mrow></math>, and <math id="S1.SS1.p2.m2" class="ltx_Math" alttext="H" display="inline"><mi>H</mi></math> depends only on temperature (Sandler, 2006).Consequently, in the subsequent development, we assume that <math id="S1.SS1.p2.m3" class="ltx_Math" alttext="U_{\rm int}=H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>=</mo><mi>H</mi></mrow></math> and <math id="S1.SS1.p2.m4" class="ltx_Math" alttext="\hat{U}_{\rm int}=\hat{H}" display="inline"><mrow><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub><mo>=</mo><mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover></mrow></math> where the caret (^) means per unit mass.As shown in Appendix C, a differential change in temperature <math id="S1.SS1.p2.m5" class="ltx_Math" alttext="dT" display="inline"><mrow><mi>d</mi><mo>⁢</mo><mi>T</mi></mrow></math>, produces a corresponding change in the internal energy per unit mass, <math id="S1.SS1.p2.m6" class="ltx_Math" alttext="d\hat{U}_{\rm int}" display="inline"><mrow><mi>d</mi><mo>⁢</mo><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub></mrow></math>,</p>
# """
x_text = """
<p class="ltx_p">Consequently, in the subsequent development, we assume that <math id="S1.SS1.p2.m3" class="ltx_Math" alttext="U_{\rm int}=H" display="inline"><mrow><msub><mi>U</mi><mi>int</mi></msub><mo>=</mo><mi>H</mi></mrow></math> and <math id="S1.SS1.p2.m4" class="ltx_Math" alttext="\hat{U}_{\rm int}=\hat{H}" display="inline"><mrow><msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub><mo>=</mo><mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover></mrow></math> where the caret (^) means per unit mass.</p>
"""

In [83]:
import copy

In [84]:
tree = lxml.html.parse(doc[0].path)
html = tree.getroot()
x_list = html.cssselect('math')

In [85]:
i, r = [], []
x_tree = lxml.html.fromstring(x_text)
# print(x_tree.text_content())
# for x_math in copy.deepcopy(x_tree).cssselect('math'):
#     print(tree_to_str(x_math))
    # print(extract_ml_component(x_math, 'mover', i, r))
# print(x_tree.text_content())
math_txt = []
identifiers = []
reg_string_list = []
mltags = ['msubsup', 'msub', 'msup', 'munderover', 'munder', 'mover']
for html_math_mltag in copy.deepcopy(x_tree).cssselect('math'):
    for mltag in mltags:
        identifiers, reg_string_list = extract_ml_component(html_math_mltag, mltag, identifiers, reg_string_list)


IndexError: string index out of range

In [80]:
identifiers

['Uxintx', '\\overset{^}{U}intx', 'Hx^x']

In [68]:
reg_string_list

[('Uxintx', '<msub><mi>U</mi><mi>int</mi></msub>', 'MATH0000'),
 ('\\overset{^}{U}intx',
  '<msub><mover accent="true"><mi>U</mi><mo stretchy="false">^</mo></mover><mi>int</mi></msub>',
  'MATH0001'),
 ('Hx^x',
  '<mover accent="true"><mi>H</mi><mo stretchy="false">^</mo></mover>',
  'MATH0002')]